<a href="https://colab.research.google.com/github/just5ebas/cripto_test/blob/main/G03_4AA_HKDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Lorem ipsum
!pip install lorem-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# HKDF
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

In [3]:
import time

In [4]:
from lorem_text import lorem

def generar_lorem(num_palabras):
    texto = lorem.words(num_palabras)
    return texto

##1. Leer un archivo con el texto del mensaje a cifrar.

In [6]:
inicio = time.time()

mensaje = generar_lorem(10).encode()

fin = time.time()

duracion = fin - inicio

print("Tiempo de ejecución:", duracion*1000, "milisegundos")

Tiempo de ejecución: 0.16307830810546875 milisegundos


##2. Generar e imprimir la(las) claves de cifrado y/o descifrado.

In [17]:
inicio = time.time()

private_key1 = ec.generate_private_key(ec.SECP256R1(), default_backend())
public_key1 = private_key1.public_key()

private_key2 = ec.generate_private_key(ec.SECP256R1(), default_backend())
public_key2 = private_key2.public_key()

shared_key1 = private_key1.exchange(ec.ECDH(), public_key2)
shared_key2 = private_key2.exchange(ec.ECDH(), public_key1)

print('Clave privada 1:',private_key1)
print('Clave publica 1:',public_key1)
print('Clave privada 2:',private_key2)
print('Clave publica 2:',public_key2)

print('Clave compartida 1:',shared_key1)
print('Clave compartida 2:',shared_key2)

fin = time.time()

duracion = fin - inicio

print("Tiempo de ejecución:", duracion*1000, "milisegundos")

Clave privada 1: <cryptography.hazmat.backends.openssl.ec._EllipticCurvePrivateKey object at 0x7f14c148df60>
Clave publica 1: <cryptography.hazmat.backends.openssl.ec._EllipticCurvePublicKey object at 0x7f14c148ecb0>
Clave privada 2: <cryptography.hazmat.backends.openssl.ec._EllipticCurvePrivateKey object at 0x7f14c148fca0>
Clave publica 2: <cryptography.hazmat.backends.openssl.ec._EllipticCurvePublicKey object at 0x7f14d9aa2c20>
Clave compartida 1: b'\xb6u7\xe3\x91[i\x9d\x15\xd6:\x8e\t\xa8\xb0\x9e}\x1d0-2\xf8-.\xff\xe1\xc3j!\x81T\xf1'
Clave compartida 2: b'\xb6u7\xe3\x91[i\x9d\x15\xd6:\x8e\t\xa8\xb0\x9e}\x1d0-2\xf8-.\xff\xe1\xc3j!\x81T\xf1'
Tiempo de ejecución: 10.318994522094727 milisegundos


##3. Cifrar e imprimir el texto.

In [14]:
inicio = time.time()

hkdf = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'session_key',
    backend=default_backend()
)
session_key = hkdf.derive(shared_key1)

nonce = os.urandom(16)
cipher = Cipher(algorithms.AES(session_key), modes.CTR(nonce), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(mensaje) + encryptor.finalize()

print('Texto cifrado:',ciphertext.hex())

fin = time.time()

duracion = fin - inicio

print("Tiempo de ejecución:", duracion*1000, "milisegundos")

5205e2c8a014714654a819d92821de6f794c7a881276afff30391e93ffa2d96f23f0a49ae52a6ab928e50fb3c41806ee40c2dd7096c73293e215f762b7b09fb6584e93cb88a2e2a1c093c0a14497f8d7
Tiempo de ejecución: 2.072572708129883 milisegundos


##4. Descifrar e imprimir el texto.

In [16]:
inicio = time.time()

decryptor = cipher.decryptor()
decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()

if decrypted_text == mensaje:
  print("Descifrado correcto")

print('Texto descifrado:',decrypted_text.decode())

fin = time.time()

duracion = fin - inicio
print("Tiempo de ejecución:", duracion*1000, "milisegundos")

Descifrado correcto
cupiditate ratione enim tenetur temporibus accusamus ex doloremque dolores nulla
Tiempo de ejecución: 2.264738082885742 milisegundos
